![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training and Reusing Clinical Named Entity Recognition Models

Please make sure that your cluster is setup properly according to https://nlp.johnsnowlabs.com/docs/en/licensed_install#install-spark-nlp-for-healthcare-on-databricks

## Blogposts and videos:

https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77

https://www.youtube.com/watch?v=YM-e4eOiQ34

https://medium.com/spark-nlp/named-entity-recognition-for-healthcare-with-sparknlp-nerdl-and-nercrf-a7751b6ad571

https://medium.com/atlas-research/ner-for-clinical-text-7c73caddd180

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 3.4.0
sparknlp_jsl.version : 3.4.0
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.0 
 Master 
 spark://10.139.64.6:7077 
 AppName 
 Databricks Shell

# Clinical NER Pipeline (with pretrained models)

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
  .setInputCols(["document"]) \
  .setOutputCol("sentence") 

#sentenceDetector = SentenceDetector()\
#   .setInputCols(["document"])\
#   .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")\
  .setLabelCasing("upper")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
#checking the stages in pipeline
model.stages

Out[3]: [DocumentAssembler_c8dae7e6491d,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_2b1925bc87a6,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_1a8637089929,
 NerConverter_813dd24b7ef5]

In [0]:
#getting the classes in pretrained model
clinical_ner.getClasses()

Out[4]: ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST']

In [0]:
#extracting the embedded default param values
clinical_ner.extractParamMap()

Out[5]: {Param(parent='MedicalNerModel_1a8637089929', name='includeAllConfidenceScores', doc='whether to include all confidence scores in annotation metadata or just the score of the predicted tag'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='inferenceBatchSize', doc='number of sentences to process in a single batch during inference'): 1,
 Param(parent='MedicalNerModel_1a8637089929', name='labelCasing', doc='Setting all labels of the NER models upper/lower case. values upper|lower'): 'upper',
 Param(parent='MedicalNerModel_1a8637089929', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='includeConfidence', doc='whether to include confidence scores in annotation metadata'): True,
 Param(parent='MedicalNerModel_1a8637089929', name='batchSize', doc='Size of every batch'): 64,
 Param(parent='MedicalNerModel_1a8637089929', name='classes', doc='get the tags used to trained this MedicalNerModel'): ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST'],
 Param(parent='MedicalNerModel_1a8637089929', name='inputCols', doc='previous annotations columns, if renamed'): ['sentence',
 'token',
 'embeddings'],
 Param(parent='MedicalNerModel_1a8637089929', name='outputCol', doc='output annotation column. can be left default.'): 'ner',
 Param(parent='MedicalNerModel_1a8637089929', name='storageRef', doc='unique reference name for identification'): 'clinical'}

In [0]:
#checking the embeddings
clinical_ner.getStorageRef()

Out[6]: 'clinical'

In [0]:
#downloading the sample dataset
! wget -q https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv

dbutils.fs.cp("file:/databricks/driver/pubmed_sample_text_small.csv", "dbfs:/") 

Out[7]: True

In [0]:
%fs ls file:/databricks/driver 

path,name,size
file:/databricks/driver/preload_class.lst,preload_class.lst,912170
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/pubmed_sample_text_small.csv,pubmed_sample_text_small.csv,9363435
file:/databricks/driver/logs/,logs/,4096
file:/databricks/driver/ganglia/,ganglia/,4096


In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("dbfs:/pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=80)

+--------------------------------------------------------------------------------+
 text|
+--------------------------------------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwar...|
BACKGROUND: At present, it is one of the most important issues for the treatm...|
OBJECTIVE: To investigate the relationship between preoperative atrialfibrill...|
Combined EEG/fMRI recording has been used to localize the generators of EEGev...|
Kohlschutter syndrome is a rare neurodegenerative disorder presenting withint...|
Statistical analysis of neuroimages is commonly approached with intergroupcom...|
The synthetic DOX-LNA conjugate was characterized by proton nuclear magneticr...|
Our objective was to compare three different methods of blood pressuremeasure...|
We conducted a phase II study to assess the efficacy and tolerability ofirino...|
"""Monomeric sarcosine oxidase (MSOX) is a flavoenzyme that catalyzes the oxi...|
We presented the tachinid fly Exorista japonica with moving host models: afre...|
The literature dealing with the water conducting properties of sapwood xylem ...|
A novel approach to synthesize chitosan-O-isopropyl-5'-O-d4T monophosphatecon...|
An HPLC-ESI-MS-MS method has been developed for the quantitative determinatio...|
The localizing and lateralizing values of eye and head ictal deviations durin...|
OBJECTIVE: To evaluate the effectiveness and acceptability of expectantmanage...|
For the construction of new combinatorial libraries, a lead compound was crea...|
We report the results of a screen for genetic association with urinary arseni...|
Intraparenchymal pericatheter cyst is rarely reported. Obstruction in thevent...|
It is known that patients with Klinefelter's syndrome are inclined to develop...|
+--------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
pubMedDF.printSchema()

root
-- text: string (nullable = true)

In [0]:
result = model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
A numerical metho...|[{document, 0, 88...|[{document, 0, 15...|[{token, 0, 0, A,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 69, 95, ...|
Recording of the ...|[{document, 0, 44...|[{document, 0, 10...|[{token, 0, 8, Re...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 13, 30, ...|
Colicin E5 cleave...|[{document, 0, 12...|[{document, 0, 11...|[{token, 0, 6, Co...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 9, Co...|
Recombinant adeno...|[{document, 0, 13...|[{document, 0, 20...|[{token, 0, 10, R...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 28, R...|
To understand its...|[{document, 0, 29...|[{document, 0, 19...|[{token, 0, 1, To...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 37, 52, ...|
OBJECTIVE: To rev...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 63, 91, ...|
AIM: Understandin...|[{document, 0, 15...|[{document, 0, 16...|[{token, 0, 2, AI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 46, 74, ...|
In the present st...|[{document, 0, 92...|[{document, 0, 21...|[{token, 0, 1, In...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 19, t...|
AIM: To describe ...|[{document, 0, 14...|[{document, 0, 27...|[{token, 0, 2, AI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 17, 32, ...|
Although the risk...|[{document, 0, 12...|[{document, 0, 11...|[{token, 0, 7, Al...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 118, 163...|
Successful long-t...|[{document, 0, 16...|[{document, 0, 12...|[{token, 0, 9, Su...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 43, S...|
Pulmonary alveola...|[{document, 0, 16...|[{document, 0, 15...|[{token, 0, 8, Pu...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 94, ...|
Prenatal care is ...|[{document, 0, 26...|[{document, 0, 68...|[{token, 0, 7, Pr...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 70, 86, ...|
Controversy exist...|[{document, 0, 12...|[{document, 0, 18...|[{token, 0, 10, C...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 43, 66, ...|
Children with a f...|[{document, 0, 34...|[{document, 0, 13...|[{token, 0, 7, Ch...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 34, 46, ...|
The mucosal immun...|[{document, 0, 19...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 51, 57, ...|
Urinary concretio...|[{document, 0, 86...|[{document, 0, 15...|[{token, 0, 6, Ur...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 18, U...|
In this study, th...|[{document, 0, 14...|[{document, 0, 17...|[{token, 0, 1, In...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 12, t...|
The presence of c...|[{document, 0, 44...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 16, 32, ...|
Phosphatidylethan...|[{document, 0, 17...|[{document, 0, 26...|[{token, 0, 23, P...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 48, P...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select('token.result','ner.result').show(truncate=80)

+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
 result| result|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
[A, numerical, method, was, developed, using, geometrical, optics, to, predic...|[O, O, O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, O, B-PROBLEM, O, O, O, O...|
[Recording, of, the, esophageal, ECG, is, important, in, differentiating, bet...|[O, O, B-TEST, I-TEST, I-TEST, O, O, O, O, O, O, O, B-PROBLEM, O, B-TREATMENT...|
[Colicin, E5, cleaves, tRNAs, for, Tyr, ,, His, ,, Asn, and, Asp, in, their, ...|[B-TREATMENT, I-TREATMENT, O, O, O, B-TREATMENT, O, O, O, B-TREATMENT, O, B-T...|
[Recombinant, adenoviruses, (, rAd, ),, widely, used, as, vectors, for, gene,...|[B-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, O, O, O, O, B-TREATMENT,...|
[To, understand, its, potential, to, cause, invasive, disease, ,, the, genome...|[O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, O, O, O, O, B-PROBLEM, I-PROBLEM, O,...|
[OBJECTIVE, :, To, review, a, group, of, Australian, women, presenting, to, a...|[O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, B-PROBLEM, O, ...|
[AIM, :, Understanding, factors, that, contribute, to, changes, in, arterial,...|[O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O,...|
[In, the, present, study, ,, the, pharmacological, effects, of, BmK, AS, ,, a...|[O, B-TEST, I-TEST, I-TEST, O, B-TREATMENT, I-TREATMENT, I-TREATMENT, O, B-PR...|
[AIM, :, To, describe, the, risk, factors, and, molecular, epidemiology, of, ...|[O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, B-PROBLEM, I-PROBLE...|
[Although, the, risks, smoking, poses, to, health, are, now, well, known, ,, ...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TEST,...|
[Successful, long-term, myocardial, preservation, is, dependent, on, optimizi...|[B-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, O, B-PROBLEM, O, B-TREAT...|
[Pulmonary, alveolar, macrophages, lavaged, from, tobacco, smokers, release, ...|[O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, O, O, O...|
[Prenatal, care, is, focused, on, improving, the, health, of, women, and, bab...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, O, O, O, O, O, O, O, O, ...|
[Controversy, exists, regarding, the, safety, of, intravenous, dipyridamole, ...|[O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, B-PROBLEM, I-PROBLEM, I-PR...|
[Children, with, a, family, history, of, thrombophilia, and/or, thrombosis, a...|[O, O, O, O, O, O, B-PROBLEM, O, B-PROBLEM, O, O, O, B-PROBLEM, I-PROBLEM, O,...|
[The, mucosal, immune, system, is, constantly, exposed, to, antigen, ,, wheth...|[O, O, O, O, O, O, O, O, B-TEST, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, O, O...|
[Urinary, concretions, ,, particularly, in, the, upper, urinary, tract, ,, oc...|[B-PROBLEM, I-PROBLEM, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROB...|
[In, this, study, ,, the, biosorption, of, nickel(II, ), ion, on, deactivated...|[O, B-TEST, I-TEST, O, O, O, O, B-TREATMENT, O, B-TREATMENT, O, B-TREATMENT, ...|
[The, presence, of, coronary, ectasias, in, otherwise, normal, epicardial, co...|[O, O, O, B-PROBLEM, I-PROBLEM, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PRO...|
[Phosphatidylethanolamine, methyltransferase, (, PEMT, ), and, phospholipidme...|[B-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, O, B-PROBLE...|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
result_df= result.select(F.explode(F.arrays_zip(result.token.result, result.ner.result)).alias("cols"))\
                 .select(F.expr("cols['0']").alias("token"),
                         F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)

+----------------+---------+
 token|ner_label|
+----------------+---------+
 A| O|
 numerical| O|
 method| O|
 was| O|
 developed| O|
 using| O|
 geometrical| O|
 optics| O|
 to| O|
 predict| O|
far-fieldoptical|B-PROBLEM|
 scattering|I-PROBLEM|
 from| O|
 particles|B-PROBLEM|
 that| O|
 are| O|
 symmetric| O|
 about| O|
 the| O|
 optic| O|
 axis| O|
 .| O|
 Thediffractive| O|
 component| O|
 of| O|
 scattering| O|
 is| O|
 calculated| O|
 and| O|
 combined| O|
 with| O|
 thereflective| O|
 and| O|
 refractive| O|
 components| O|
 to| O|
 give| O|
 the| O|
 total| O|
 scattering| O|
 pattern| O|
 .| O|
 Thephase| O|
 terms| O|
 of| O|
 the| O|
 scattered| O|
 light| O|
 are| O|
 calculated| O|
+----------------+---------+
only showing top 50 rows

In [0]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+-----------+-----+
ner_label |count|
+-----------+-----+
O |13802|
I-PROBLEM |1638 |
B-PROBLEM |928 |
I-TREATMENT|756 |
B-TREATMENT|629 |
I-TEST |624 |
B-TEST |519 |
+-----------+-----+

In [0]:
result.select('ner_chunk').take(1)

Out[16]: [Row(ner_chunk=[Row(annotatorType='chunk', begin=69, end=95, result='far-fieldoptical scattering', metadata={'sentence': '0', 'chunk': '0', 'entity': 'PROBLEM', 'confidence': '0.95575'}, embeddings=[]), Row(annotatorType='chunk', begin=102, end=110, result='particles', metadata={'sentence': '0', 'chunk': '1', 'entity': 'PROBLEM', 'confidence': '0.9978'}, embeddings=[]), Row(annotatorType='chunk', begin=380, end=388, result='themethod', metadata={'sentence': '3', 'chunk': '2', 'entity': 'TREATMENT', 'confidence': '0.9982'}, embeddings=[]), Row(annotatorType='chunk', begin=415, end=434, result='a spherical particle', metadata={'sentence': '3', 'chunk': '3', 'entity': 'PROBLEM', 'confidence': '0.8060333'}, embeddings=[]), Row(annotatorType='chunk', begin=462, end=484, result='toMie scattering theory', metadata={'sentence': '3', 'chunk': '4', 'entity': 'TREATMENT', 'confidence': '0.7798'}, embeddings=[]), Row(annotatorType='chunk', begin=576, end=603, result='small-amplitude oscillations', metadata={'sentence': '5', 'chunk': '5', 'entity': 'PROBLEM', 'confidence': '0.4913'}, embeddings=[]), Row(annotatorType='chunk', begin=653, end=672, result='the numerical method', metadata={'sentence': '5', 'chunk': '6', 'entity': 'TREATMENT', 'confidence': '0.74543333'}, embeddings=[]), Row(annotatorType='chunk', begin=675, end=713, result='Numerical data fromspheroidal particles', metadata={'sentence': '6', 'chunk': '7', 'entity': 'PROBLEM', 'confidence': '0.81762505'}, embeddings=[]), Row(annotatorType='chunk', begin=719, end=741, result='hemispherical particles', metadata={'sentence': '6', 'chunk': '8', 'entity': 'PROBLEM', 'confidence': '0.99475'}, embeddings=[]), Row(annotatorType='chunk', begin=771, end=795, result='ofhemispherical particles', metadata={'sentence': '7', 'chunk': '9', 'entity': 'PROBLEM', 'confidence': '0.84860003'}, embeddings=[]), Row(annotatorType='chunk', begin=800, end=812, result='a calibration', metadata={'sentence': '7', 'chunk': '10', 'entity': 'TEST', 'confidence': '0.9699'}, embeddings=[]), Row(annotatorType='chunk', begin=827, end=875, result='intensity-type opticalparticle-sizing instruments', metadata={'sentence': '7', 'chunk': '11', 'entity': 'TREATMENT', 'confidence': '0.64886665'}, embeddings=[])])]

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------------------------------------+---------+
chunk |ner_label|
+-----------------------------------------------------------+---------+
the G-protein-activated inwardly rectifying potassium (GIRK|TREATMENT|
the genomicorganization |TREATMENT|
a candidate gene forType II diabetes mellitus |PROBLEM |
byapproximately |TREATMENT|
single nucleotide polymorphisms |TREATMENT|
aVal366Ala substitution |PROBLEM |
an 8 base-pair |PROBLEM |
insertion/deletion |PROBLEM |
Ourexpression studies |TEST |
the transcript in various humantissues |PROBLEM |
fat andskeletal muscle |PROBLEM |
furtherstudies |PROBLEM |
the KCNJ9 protein |TREATMENT|
evaluation |TEST |
Type II diabetes |PROBLEM |
the treatment |TREATMENT|
breast cancer |PROBLEM |
the standard therapy |TREATMENT|
anthracyclines |TREATMENT|
taxanes |TREATMENT|
+-----------------------------------------------------------+---------+
only showing top 20 rows

In [0]:
# fullAnnotate in LightPipeline

text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .
'''

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

df_clinical = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df_clinical.head(20)

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .

Out[18]:

,chunks,begin,end,sentence_id,entities
0,gestational diabetes mellitus,40,68,0,PROBLEM
1,subsequent type two diabetes mellitus,118,154,0,PROBLEM
2,T2DM,158,161,0,PROBLEM
3,HTG-induced pancreatitis,187,210,0,PROBLEM
4,an acute hepatitis,264,281,0,PROBLEM
5,obesity,289,295,0,PROBLEM
6,a body mass index,302,318,0,PROBLEM
7,BMI,322,324,0,TEST
8,polyuria,381,388,0,PROBLEM
9,polydipsia,392,401,0,PROBLEM


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

# Change color of an entity label

#visualiser.set_label_colors({'PROBLEM':'#008080', 'TEST':'#800080', 'TREATMENT':'#808080'})
#visualiser.display(light_result[0], label_col='ner_chunk')

# Set label filter

# visualiser.display(light_result, label_col='ner_chunk', document_col='document',
                   #labels=['PROBLEM','TEST'])
  
displayHTML(ner_vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM ( T2DM PROBLEM ), one prior episode of HTG-induced pancreatitis PROBLEM three years prior to presentation , associated with an acute hepatitis PROBLEM , and obesity PROBLEM with a body mass index PROBLEM ( BMI TEST ) of 33.5 kg/m2 , presented with a one-week history of polyuria PROBLEM , polydipsia PROBLEM , poor appetite PROBLEM , and vomiting PROBLEM . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin TREATMENT for a respiratory tract infection PROBLEM . She was on metformin TREATMENT , glipizide TREATMENT , and dapagliflozin TREATMENT for T2DM PROBLEM and atorvastatin TREATMENT and gemfibrozil TREATMENT for HTG PROBLEM . She had been on dapagliflozin TREATMENT for six months at the time of presentation . Physical examination TEST on presentation was significant for dry oral mucosa PROBLEM ; significantly , her abdominal examination TEST was benign with no tenderness PROBLEM , guarding PROBLEM , or rigidity PROBLEM . Pertinent laboratory findings on admission were : serum glucose TEST 111 mg/dl , bicarbonate TEST 18 mmol/l , anion gap TEST 20 , creatinine TEST 0.4 mg/dL , triglycerides TEST 508 mg/dL , total cholesterol TEST 122 mg/dL , glycated hemoglobin TEST ( HbA1c TEST ) 10% , and venous pH TEST 7.27 . Serum lipase TEST was normal at 43 U/L TEST . Serum acetone levels TEST could not be assessed as blood samples TEST kept hemolyzing PROBLEM due to significant lipemia PROBLEM . The patient was initially admitted for starvation ketosis PROBLEM , as she reported poor oral intake PROBLEM for three days prior to admission . However , serum chemistry TEST obtained six hours after presentation revealed her glucose TEST was 186 mg/dL , the anion gap TEST was still elevated PROBLEM at 21 , serum bicarbonate TEST was 16 mmol/L , triglyceride level TEST peaked at 2050 mg/dL , and lipase TEST was 52 U/L TEST . The β-hydroxybutyrate level TEST was obtained and found to be elevated PROBLEM at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis TEST due to interference from turbidity PROBLEM caused by lipemia PROBLEM again . The patient was treated with an insulin drip TREATMENT for euDKA PROBLEM and HTG PROBLEM with a reduction TREATMENT in the anion gap TEST to 13 and triglycerides TEST to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection PROBLEM in the setting of SGLT2 inhibitor TREATMENT use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine TREATMENT at night , 12 units of insulin lispro TREATMENT with meals , and metformin TREATMENT 1000 mg two times a day . It was determined that all SGLT2 inhibitors TREATMENT should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .

## NER JSL Model

Let's show an example of `ner_jsl` model that has about 80 labels by changing just only the model name.

**Entities**

| | | | | |
|-|-|-|-|-|
|Injury_or_Poisoning|Direction|Test|Admission_Discharge|Death_Entity|
|Relationship_Status|Duration|Respiration|Hyperlipidemia|Birth_Entity|
|Age|Labour_Delivery|Family_History_Header|BMI|Temperature|
|Alcohol|Kidney_Disease|Oncological|Medical_History_Header|Cerebrovascular_Disease|
|Oxygen_Therapy|O2_Saturation|Psychological_Condition|Heart_Disease|Employment|
|Obesity|Disease_Syndrome_Disorder|Pregnancy|ImagingFindings|Procedure|
|Medical_Device|Race_Ethnicity|Section_Header|Symptom|Treatment|
|Substance|Route|Drug_Ingredient|Blood_Pressure|Diet|
|External_body_part_or_region|LDL|VS_Finding|Allergen|EKG_Findings|
|Imaging_Technique|Triglycerides|RelativeTime|Gender|Pulse|
|Social_History_Header|Substance_Quantity|Diabetes|Modifier|Internal_organ_or_component|
|Clinical_Dept|Form|Drug_BrandName|Strength|Fetus_NewBorn|
|RelativeDate|Height|Test_Result|Sexually_Active_or_Sexual_Orientation|Frequency|
|Time|Weight|Vaccine|Vital_Signs_Header|Communicable_Disease|
|Dosage|Overweight|Hypertension|HDL|Total_Cholesterol|
|Smoking|Date||||

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
  .setInputCols(["document"]) \
  .setOutputCol("sentence") 

#sentenceDetector = SentenceDetector()\
#   .setInputCols(["document"])\
#   .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
jsl_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("jsl_ner")\
  .setLabelCasing("upper")

jsl_ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "jsl_ner"]) \
  .setOutputCol("jsl_ner_chunk")

jsl_ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

jsl_ner_model = jsl_ner_pipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][OK!]

In [0]:
print (text)

jsl_light_model = LightPipeline(jsl_ner_model)

jsl_light_result = jsl_light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in jsl_light_result[0]['jsl_ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

jsl_df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

jsl_df.head(20)

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .

Out[21]:

,chunks,begin,end,sentence_id,entities
0,28-year-old,3,13,0,AGE
1,female,15,20,0,GENDER
2,gestational diabetes mellitus,40,68,0,DIABETES
3,eight years prior,80,96,0,RELATIVEDATE
4,subsequent,118,127,0,MODIFIER
5,type two diabetes mellitus,129,154,0,DIABETES
6,T2DM,158,161,0,DIABETES
7,HTG-induced pancreatitis,187,210,0,DISEASE_SYNDROME_DISORDER
8,three years prior,212,228,0,RELATIVEDATE
9,acute,267,271,0,MODIFIER


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(jsl_light_result[0], label_col='jsl_ner_chunk', document_col='document', return_html=True)

displayHTML(ner_vis)

A 28-year-old AGE female GENDER with a history of gestational diabetes mellitus DIABETES diagnosed eight years prior RELATIVEDATE to presentation and subsequent MODIFIER type two diabetes mellitus DIABETES ( T2DM DIABETES ), one prior episode of HTG-induced pancreatitis DISEASE_SYNDROME_DISORDER three years prior RELATIVEDATE to presentation , associated with an acute MODIFIER hepatitis DISEASE_SYNDROME_DISORDER , and obesity OBESITY with a body mass index SYMPTOM ( BMI ) of 33.5 kg/m2 WEIGHT , presented with a one-week DURATION history of polyuria SYMPTOM , polydipsia SYMPTOM , poor appetite SYMPTOM , and vomiting SYMPTOM . Two weeks prior RELATIVEDATE to presentation , she GENDER was treated with a five-day FREQUENCY course of amoxicillin DRUG_INGREDIENT for a respiratory tract infection DISEASE_SYNDROME_DISORDER . She GENDER was on metformin DRUG_INGREDIENT , glipizide DRUG_INGREDIENT , and dapagliflozin DRUG_INGREDIENT for T2DM DRUG_INGREDIENT and atorvastatin DRUG_INGREDIENT and gemfibrozil DRUG_INGREDIENT for HTG DRUG_INGREDIENT . She GENDER had been on dapagliflozin DRUG_INGREDIENT for six months DURATION at the time of presentation . Physical examination on presentation was significant for dry oral mucosa SYMPTOM ; significantly , her GENDER abdominal EXTERNAL_BODY_PART_OR_REGION examination was benign with no tenderness SYMPTOM , guarding SYMPTOM , or rigidity SYMPTOM . Pertinent laboratory findings on admission ADMISSION_DISCHARGE were : serum glucose TEST 111 mg/dl TEST_RESULT , bicarbonate TEST 18 mmol/l TEST_RESULT , anion gap TEST 20 TEST_RESULT , creatinine TEST 0.4 mg/dL TEST_RESULT , triglycerides 508 mg/dL TRIGLYCERIDES , total cholesterol TEST 122 mg/dL TEST_RESULT , glycated hemoglobin TEST ( HbA1c TEST ) 10% TEST_RESULT , and venous pH TEST 7.27 TEST_RESULT . Serum lipase TEST was normal TEST_RESULT at 43 U/L TEST_RESULT . Serum acetone levels TEST could not be assessed as blood samples TEST kept hemolyzing TEST_RESULT due to significant MODIFIER lipemia SYMPTOM . The patient was initially admitted ADMISSION_DISCHARGE for starvation ketosis DISEASE_SYNDROME_DISORDER , as she GENDER reported poor oral intake SYMPTOM for three days DURATION prior to admission ADMISSION_DISCHARGE . However , serum chemistry TEST obtained six hours RELATIVETIME after presentation revealed her GENDER glucose TEST was 186 mg/dL TEST_RESULT , the anion gap TEST was still elevated TEST_RESULT at 21 TEST_RESULT , serum bicarbonate TEST was 16 mmol/L TEST_RESULT , triglyceride level TEST peaked at 2050 mg/dL TEST_RESULT , and lipase TEST was 52 U/L TEST_RESULT . The β-hydroxybutyrate level TEST was obtained and found to be elevated TEST_RESULT at 5.29 mmol/L TEST_RESULT - the original sample was centrifuged and the chylomicron layer INTERNAL_ORGAN_OR_COMPONENT removed prior to analysis due to interference from turbidity SYMPTOM caused by lipemia SYMPTOM again . The patient was treated with an insulin DRUG_INGREDIENT drip ROUTE for euDKA and HTG DRUG_INGREDIENT with a reduction in the anion gap to 13 TEST_RESULT and triglycerides TEST to 1400 mg/dL TEST_RESULT , within 24 hours RELATIVETIME . Her GENDER euDKA was thought to be precipitated by her GENDER respiratory tract infection DISEASE_SYNDROME_DISORDER in the setting of SGLT2 inhibitor DRUG_INGREDIENT use . The patient was seen by the endocrinology service CLINICAL_DEPT and she GENDER was discharged ADMISSION_DISCHARGE on 40 units DOSAGE of insulin glargine DRUG_INGREDIENT at night FREQUENCY , 12 units DOSAGE of insulin lispro DRUG_INGREDIENT with meals FREQUENCY , and metformin DRUG_INGREDIENT 1000 mg STRENGTH two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG_INGREDIENT should be discontinued indefinitely . She GENDER had close follow-up with endocrinology CLINICAL_DEPT post discharge ADMISSION_DISCHARGE .

## Posology NER

In [0]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    .setLabelCasing("upper")

ner_converter = NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

# greedy model
posology_ner_greedy = MedicalNerModel.pretrained("ner_posology_greedy", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_greedy")

ner_converter_greedy = NerConverter()\
    .setInputCols(["sentence","token","ner_greedy"])\
    .setOutputCol("ner_chunk_greedy")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter,
    posology_ner_greedy,
    ner_converter_greedy])

empty_data = spark.createDataFrame([[""]]).toDF("text")
posology_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][OK!]
ner_posology_greedy download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]

In [0]:
posology_ner.getClasses()

Out[24]: ['O',
 'B-DOSAGE',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-ROUTE',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'B-DRUG',
 'I-DRUG',
 'B-FORM',
 'I-DOSAGE',
 'B-DURATION',
 'I-DURATION',
 'I-FORM',
 'I-ROUTE']

In [0]:
posology_result = posology_model.transform(pubMedDF.limit(100))

In [0]:
posology_result.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| ner_greedy| ner_chunk_greedy|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
A numerical metho...|[{document, 0, 88...|[{document, 0, 15...|[{token, 0, 0, A,...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Recording of the ...|[{document, 0, 44...|[{document, 0, 10...|[{token, 0, 8, Re...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Colicin E5 cleave...|[{document, 0, 12...|[{document, 0, 11...|[{token, 0, 6, Co...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Recombinant adeno...|[{document, 0, 13...|[{document, 0, 20...|[{token, 0, 10, R...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 539, 554...|[{named_entity, 0...|[{chunk, 539, 554...|
To understand its...|[{document, 0, 29...|[{document, 0, 19...|[{token, 0, 1, To...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
OBJECTIVE: To rev...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 972, 100...|[{named_entity, 0...|[{chunk, 65, 91, ...|
AIM: Understandin...|[{document, 0, 15...|[{document, 0, 16...|[{token, 0, 2, AI...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
In the present st...|[{document, 0, 92...|[{document, 0, 21...|[{token, 0, 1, In...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 53, 55, ...|[{named_entity, 0...|[{chunk, 634, 636...|
AIM: To describe ...|[{document, 0, 14...|[{document, 0, 27...|[{token, 0, 2, AI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 975, 100...|[{named_entity, 0...|[{chunk, 226, 263...|
Although the risk...|[{document, 0, 12...|[{document, 0, 11...|[{token, 0, 7, Al...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 10 rows

In [0]:
posology_result.printSchema()

root
-- text: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_greedy: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk_greedy: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + id
posology_result = posology_result.withColumn("id", monotonically_increasing_id())

posology_result.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
 text| document| sentence| token| embeddings| ner| ner_chunk| ner_greedy| ner_chunk_greedy| id|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
The human KCNJ9 (...|[{document, 0, 95...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...|[{chunk, 52, 122,...| 0|
BACKGROUND: At pr...|[{document, 0, 14...|[{document, 0, 19...|[{token, 0, 9, BA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 167, 180...|[{named_entity, 0...|[{chunk, 167, 180...| 1|
OBJECTIVE: To inv...|[{document, 0, 15...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []| 2|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
only showing top 3 rows

In [0]:
posology_result.select('token.result','ner.result').show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[A, numerical, method, was, developed, using, geometrical, optics, to, predict, far-fieldoptical,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Recording, of, the, esophageal, ECG, is, important, in, differentiating, between, differenttypes...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Colicin, E5, cleaves, tRNAs, for, Tyr, ,, His, ,, Asn, and, Asp, in, their, anticodons, to, abol...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Recombinant, adenoviruses, (, rAd, ),, widely, used, as, vectors, for, gene, therapy, ,, aregene...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[To, understand, its, potential, to, cause, invasive, disease, ,, the, genome, of, Mycoplasmacani...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[OBJECTIVE, :, To, review, a, group, of, Australian, women, presenting, to, a, reproductiveendocr...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[AIM, :, Understanding, factors, that, contribute, to, changes, in, arterial, stiffness, over, ti...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[In, the, present, study, ,, the, pharmacological, effects, of, BmK, AS, ,, a, beta-like, scorpio...|[O, O, O, O, O, O, O, O, O, B-DRUG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
[AIM, :, To, describe, the, risk, factors, and, molecular, epidemiology, of, nosocomialbloodstrea...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Although, the, risks, smoking, poses, to, health, are, now, well, known, ,, many, young, people,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Successful, long-term, myocardial, preservation, is, dependent, on, optimizingconditions, during...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Pulmonary, alveolar, macrophages, lavaged, from, tobacco, smokers, release, increasedlevels, of,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Prenatal, care, is, focused, on, improving, the, health, of, women, and, babies, ., Traditionalm...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Controversy, exists, regarding, the, safety, of, intravenous, dipyridamole, in, patientswith, se...|[O, O, O, O, O, O, B-ROUTE, B-DRUG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
[Children, with, a, family, history, of, thrombophilia, and/or, thrombosis, are, oftenreferred, t...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[The, mucosal, immune, system, is, constantly, exposed, to, antigen, ,, whether, it, be, foodanti...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Urinary, concretions, ,, particularly, in, the, upper, urinary, tract, ,, occur, in, otherwise, ...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[In, this, study, ,, the, biosorption, of, nickel(II, ), ion, on, deactivated, protonated, yeast,...|[O, O, O, O, O, O, O, O

In [0]:
from pyspark.sql import functions as F

posology_result_df = posology_result.select(F.explode(F.arrays_zip(posology_result.token.result, posology_result.ner.result )).alias("cols")) \
                                    .select(F.expr("cols['0']").alias("token"),
                                            F.expr("cols['1']").alias("ner_label"))\
                                    .filter("ner_label!='O'")

posology_result_df.show(20, truncate=100)

+---------------+-----------+
 token| ner_label|
+---------------+-----------+
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 vinorelbine| B-DRUG|
 vinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 Vinorelbinewas| B-DRUG|
 25| B-STRENGTH|
 mg/m(2| I-STRENGTH|
 intravenously| B-ROUTE|
 on|B-FREQUENCY|
 days|I-FREQUENCY|
thatvinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 DOX| B-DRUG|
 trandolapril| B-DRUG|
 losartan| B-DRUG|
 3-week| B-DURATION|
 placebo| B-DRUG|
+---------------+-----------+
only showing top 20 rows

In [0]:
posology_greedy_result_df = posology_result.select(F.explode(F.arrays_zip(posology_result.token.result,
                                                                          posology_result.ner_greedy.result)).alias("cols")) \
                                           .select(F.expr("cols['0']").alias("token"),
                                                    F.expr("cols['1']").alias("ner_label"))\
                                           .filter("ner_label!='O'")

posology_greedy_result_df.show(20, truncate=100)

+---------------------------+---------+
 token|ner_label|
+---------------------------+---------+
 lackedprotein| B-DRUG|
 VI| I-DRUG|
 immobilized| B-DRUG|
 zinc| I-DRUG|
 affinity| I-DRUG|
 chromatography| I-DRUG|
 valuabletechnique| B-DRUG|
reproductiveendocrinologist| B-DRUG|
 dehydroepiandrosterone| B-DRUG|
 sulphate| I-DRUG|
 low-dose| B-DRUG|
 dexamethasone| I-DRUG|
 withethinyloestradiol| B-DRUG|
 cyproterone| B-DRUG|
 acetate| I-DRUG|
 ethinyloestradiol| B-DRUG|
 cyproterone| B-DRUG|
 acetate| I-DRUG|
 withethinyloestradiol| B-DRUG|
 cyproterone| B-DRUG|
+---------------------------+---------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip(posology_result.ner_chunk.result, posology_result.ner_chunk.begin, posology_result.ner_chunk.end, 
                                                   posology_result.ner_chunk.metadata)).alias("cols")) \
                                      .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
                                                    F.expr("cols['0']").alias("chunk"),
                                                    F.expr("cols['1']").alias("begin"),
                                                    F.expr("cols['2']").alias("end"),
                                                    F.expr("cols['3']['entity']").alias("ner_label"))\
                                                    .filter("ner_label!='O'")\
                                      .show(truncate=False)

+---+-----------+------------------------------------------------------------------------+-----+----+---------+
id |sentence_id|chunk |begin|end |ner_label|
+---+-----------+------------------------------------------------------------------------+-----+----+---------+
3 |4 |lackedprotein VI |539 |554 |DRUG |
5 |8 |dehydroepiandrosterone sulphate |972 |1002|DRUG |
5 |8 |low-dose |1021 |1028|DOSAGE |
5 |8 |dexamethasone |1030 |1042|DRUG |
5 |9 |withethinyloestradiol |1077 |1097|DRUG |
5 |9 |cyproterone acetate |1103 |1121|DRUG |
5 |13 |ethinyloestradiol |1556 |1572|DRUG |
5 |13 |cyproterone acetate |1578 |1596|DRUG |
5 |17 |withethinyloestradiol |1914 |1934|DRUG |
5 |17 |cyproterone acetate |1940 |1958|DRUG |
7 |0 |BmK |53 |55 |DRUG |
7 |4 |BmK |634 |636 |DRUG |
8 |6 |ofbroad-spectrum cephalosporins |975 |1005|DRUG |
8 |7 |imipenem |1046 |1053|DRUG |
10 |1 |blood-perfused |258 |271 |DRUG |
10 |1 |isolatedcircuit |297 |311 |DRUG |
10 |3 |aspartate/glutamate-enriched leukocyte-depleted blood cardioplegia(group|475 |546 |DRUG |
10 |3 |andaspartate/glutamate-enriched blood cardioplegia |603 |652 |DRUG |
11 |5 |Superoxide |678 |687 |DRUG |
11 |5 |monoxide |930 |937 |DRUG |
+---+-----------+------------------------------------------------------------------------+-----+----+---------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip(posology_result.ner_chunk_greedy.result, posology_result.ner_chunk_greedy.begin, 
                                                   posology_result.ner_chunk_greedy.end, posology_result.ner_chunk_greedy.metadata)).alias("cols")) \
                                                  .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
                                                                F.expr("cols['0']").alias("chunk"),
                                                                F.expr("cols['1']").alias("begin"),
                                                                F.expr("cols['2']").alias("end"),
                                                                F.expr("cols['3']['entity']").alias("ner_label"))\
                                                                .filter("ner_label!='O'")\
                                                                .show(truncate=False)

+---+-----------+-----------------------------------------------------------------------+-----+----+---------+
id |sentence_id|chunk |begin|end |ner_label|
+---+-----------+-----------------------------------------------------------------------+-----+----+---------+
0 |0 |G-protein-activated inwardly rectifying potassium (GIRK) channel family|52 |122 |DRUG |
1 |0 |anthracyclines |167 |180 |DRUG |
1 |0 |taxanes |186 |192 |DRUG |
1 |1 |usefulnessof vinorelbine |233 |256 |DRUG |
1 |1 |vinorelbine |386 |396 |DRUG |
1 |1 |anthracyclines |433 |446 |DRUG |
1 |1 |taxanes |452 |458 |DRUG |
1 |2 |25 mg/m(2 |517 |525 |STRENGTH |
1 |2 |intravenously |528 |540 |ROUTE |
1 |2 |on days 1 and 8 of a3 week |542 |567 |FREQUENCY|
1 |9 |Themajor |1110 |1117|DRUG |
1 |10 |thatvinorelbine |1263 |1277|DRUG |
1 |10 |anthracyclines |1380 |1393|DRUG |
1 |10 |taxanes |1399 |1405|DRUG |
6 |2 |tetrazolium dye |432 |446 |DRUG |
6 |3 |DOX |576 |578 |DRUG |
7 |0 |trandolapril |174 |185 |DRUG |
7 |0 |losartan |191 |198 |DRUG |
7 |2 |placebo |320 |326 |DRUG |
7 |2 |mgtrandolapril |359 |372 |DRUG |
+---+-----------+-----------------------------------------------------------------------+-----+----+---------+
only showing top 20 rows

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].result

Out[34]: 'anthracyclines'

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].metadata

Out[35]: {'sentence': '0', 'chunk': '0', 'entity': 'DRUG', 'confidence': '0.9996'}

### with LightPipelines

In [0]:
light_model = LightPipeline(posology_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .'

light_result = light_model.annotate(text)

list(zip(light_result['token'], light_result['ner']))

Out[36]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('for', 'B-DURATION'),
 ('3', 'I-DURATION'),
 ('months', 'I-DURATION'),
 ('.', 'O')]

In [0]:
list(zip(light_result['token'], light_result['ner_greedy']))

Out[37]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DRUG'),
 ('capsule', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('Advil', 'I-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DRUG'),
 ('units', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('insulin', 'I-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DRUG'),
 ('units', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('insulin', 'I-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'I-DRUG'),
 ('mg', 'I-DRUG'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('for', 'B-DURATION'),
 ('3', 'I-DURATION'),
 ('months', 'I-DURATION'),
 ('.', 'O')]

In [0]:
light_result = light_model.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                   'begin': begin, 'end': end})

df

Out[38]:

,chunks,entities,begin,end
0,1,DOSAGE,27,27
1,capsule,FORM,29,35
2,Advil,DRUG,40,44
3,for 5 days,DURATION,46,55
4,40 units,DOSAGE,126,133
5,insulin glargine,DRUG,138,153
6,at night,FREQUENCY,155,162
7,12 units,DOSAGE,166,173
8,insulin lispro,DRUG,178,191
9,with meals,FREQUENCY,193,202


#### NER Visualization

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)
  
displayHTML(ner_vis)

The patient was prescribed 1 DOSAGE capsule FORM of Advil DRUG for 5 days DURATION . He was seen by the endocrinology service and she was discharged on 40 units DOSAGE of insulin glargine DRUG at night FREQUENCY , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG 1000 mg STRENGTH two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG should be discontinued indefinitely for 3 months DURATION .

In [0]:
# ner_greedy

visualiser_greedy = NerVisualizer()

ner_greedy_vis = visualiser_greedy.display(light_result[0], label_col='ner_chunk_greedy', document_col='document', return_html=True)

displayHTML(ner_greedy_vis)

The patient was prescribed 1 capsule of Advil DRUG for 5 days DURATION . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine DRUG at night FREQUENCY , 12 units of insulin lispro DRUG with meals FREQUENCY , and metformin 1000 mg DRUG two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG should be discontinued indefinitely for 3 months DURATION .

## Writing a generic NER function

In [0]:
def get_base_pipeline (embeddings = 'embeddings_clinical'):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  base_pipeline = Pipeline(stages=[
                    documentAssembler,
                    sentenceDetector,
                    tokenizer,
                    word_embeddings
                  ])

  return base_pipeline



def get_clinical_entities (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  loaded_ner_model = MedicalNerModel.pretrained(model_name, "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

  ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

  base_pipeline = get_base_pipeline (embeddings)

  nlpPipeline = Pipeline(stages=[
      base_pipeline,
      loaded_ner_model,
      ner_converter])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  result = model.transform(spark_df.limit(nrows))

  result = result.withColumn("id", monotonically_increasing_id())

  result_df = result.select('id',F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.begin, result.ner_chunk.end, 
                                                        result.ner_chunk.metadata)).alias("cols")) \
                    .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
                            F.expr("cols['0']").alias("chunk"),
                            F.expr("cols['1']").alias("begin"),
                            F.expr("cols['2']").alias("end"),
                            F.expr("cols['3']['entity']").alias("ner_label"))\
                            .filter("ner_label!='O'")

  return result_df

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+--------------------+-----+---+---------+
 id|sentence_id| chunk|begin|end|ner_label|
+---+-----------+--------------------+-----+---+---------+
 0| 0|the G-protein-act...| 48|106|TREATMENT|
 0| 1|the genomicorgani...| 142|164|TREATMENT|
 0| 1|a candidate gene ...| 210|254| PROBLEM|
 0| 2| byapproximately| 380|394|TREATMENT|
 0| 3|single nucleotide...| 464|494|TREATMENT|
 0| 3|aVal366Ala substi...| 532|554| PROBLEM|
 0| 3| an 8 base-pair| 561|574| PROBLEM|
 0| 4| insertion/deletion| 581|598| PROBLEM|
 0| 5|Ourexpression stu...| 601|621| TEST|
 0| 5|the transcript in...| 648|685| PROBLEM|
 0| 5|fat andskeletal m...| 749|770| PROBLEM|
 0| 6| furtherstudies| 830|843| PROBLEM|
 0| 6| the KCNJ9 protein| 864|880|TREATMENT|
 0| 6| evaluation| 892|901| TEST|
 0| 6| Type II diabetes| 940|955| PROBLEM|
 1| 0| the treatment| 67| 79|TREATMENT|
 1| 0| breast cancer| 84| 96| PROBLEM|
 1| 0|the standard therapy| 109|128|TREATMENT|
 1| 0| anthracyclines| 167|180|TREATMENT|
 1| 0| taxanes| 186|192|TREATMENT|
+---+-----------+--------------------+-----+---+---------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+--------------------+-----+----+---------+
 id|sentence_id| chunk|begin| end|ner_label|
+---+-----------+--------------------+-----+----+---------+
 3| 3| lackedprotein VI| 539| 554| DRUG|
 5| 8|dehydroepiandrost...| 972|1002| DRUG|
 5| 8| low-dose| 1021|1028| DOSAGE|
 5| 8| dexamethasone| 1030|1042| DRUG|
 5| 9|withethinyloestra...| 1077|1097| DRUG|
 5| 9| cyproterone acetate| 1103|1121| DRUG|
 5| 15| ethinyloestradiol| 1556|1572| DRUG|
 5| 15| cyproterone acetate| 1578|1596| DRUG|
 5| 18|withethinyloestra...| 1914|1934| DRUG|
 5| 18| cyproterone acetate| 1940|1958| DRUG|
 7| 0| BmK| 53| 55| DRUG|
 7| 3| BmK| 634| 636| DRUG|
 8| 5|ofbroad-spectrum ...| 975|1005| DRUG|
 10| 1| blood-perfused| 258| 271| DRUG|
 10| 1| isolatedcircuit| 297| 311| DRUG|
 10| 3|aspartate/glutama...| 475| 546| DRUG|
 10| 3|andaspartate/glut...| 603| 652| DRUG|
 11| 5| Superoxide| 678| 687| DRUG|
 11| 5| monoxide| 930| 937| DRUG|
 12| 2| modelsin| 237| 244| DRUG|
+---+-----------+--------------------+-----+----+---------+
only showing top 20 rows

In [0]:
import pandas as pd

def get_clinical_entities_light (light_model, text):

  light_result = light_model.fullAnnotate(text)

  chunks = []
  entities = []

  for n in light_result[0]['ner_chunk']:
          
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      
  df = pd.DataFrame({'chunks':chunks, 'entities':entities})

  return df

In [0]:
text ='The patient was prescribed 1 capsule of Parol with meals . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_model = LightPipeline(posology_model)

get_clinical_entities_light (light_model, text)

Out[46]:

,chunks,entities
0,1,DOSAGE
1,capsule,FORM
2,Parol,DRUG
3,with meals,FREQUENCY
4,40 units,DOSAGE
5,insulin glargine,DRUG
6,at night,FREQUENCY
7,12 units,DOSAGE
8,insulin lispro,DRUG
9,with meals,FREQUENCY


## PHI NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_deid_large'

# deidentify_dl
# ner_deid_large

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

pd_ner_df = ner_df.toPandas()

ner_deid_large download started this may take some time.
Approximate size to download 14.1 MB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
pd_ner_df.sample(20)

Out[48]:

,id,sentence_id,chunk,begin,end,ner_label
29,31,1,February 2004,178,190,DATE
46,47,5,USchildren.,936,946,NAME
19,24,8,Swedish,1350,1356,LOCATION
41,44,3,January1st,458,467,LOCATION
76,77,2,Hillsborough County,380,398,LOCATION
22,27,0,capital city,156,167,LOCATION
63,59,1,series of blended,214,230,PROFESSION
32,37,10,(2005,1829,1833,DATE
25,27,5,2024,816,819,DATE
7,24,0,dental education,122,137,PROFESSION


In [0]:
pd_ner_df.ner_label.value_counts()

Out[49]: LOCATION 52
NAME 26
DATE 14
AGE 4
PROFESSION 4
Name: ner_label, dtype: int64

## BioNLP (Cancer Genetics) NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+----------------+-----+----+----------------------+
id |sentence_id|chunk |begin|end |ner_label |
+---+-----------+----------------+-----+----+----------------------+
0 |0 |optic axis |141 |150 |Multi-tissue_structure|
1 |0 |esophageal ECG |17 |30 |Multi-tissue_structure|
1 |2 |nose |214 |217 |Organ |
1 |3 |esophageal ECG |273 |286 |Multi-tissue_structure|
2 |0 |E5 |8 |9 |Gene_or_gene_product |
2 |0 |Tyr |29 |31 |Amino_acid |
2 |0 |Asn |39 |41 |Amino_acid |
2 |0 |Asp |47 |49 |Amino_acid |
2 |0 |Escherichia coli|102 |117 |Organism |
2 |2 |RNases Aand T1 |382 |395 |Gene_or_gene_product |
2 |2 |E5-CRD |398 |403 |Gene_or_gene_product |
2 |2 |usualRNases |463 |473 |Gene_or_gene_product |
2 |4 |RNase |718 |722 |Gene_or_gene_product |
2 |4 |Lys25 |795 |799 |Simple_chemical |
2 |4 |Gln29 |802 |806 |Simple_chemical |
2 |4 |Lys60 |812 |816 |Simple_chemical |
2 |5 |Ile94 |880 |884 |Gene_or_gene_product |
2 |5 |Arg33 |919 |923 |Gene_or_gene_product |
2 |6 |Arg33 |1081 |1085|Gene_or_gene_product |
2 |7 |Lys60 |1241 |1245|Simple_chemical |
+---+-----------+----------------+-----+----+----------------------+
only showing top 20 rows

# NER Chunker
We can extract phrases that fits into a known pattern using the NER tags. NerChunker would be quite handy to extract entity groups with neighboring tokens when there is no pretrained NER model to address certain issues. Lets say we want to extract clinical findings and body parts together as a single chunk even if there are some unwanted tokens between.

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_chunker = NerChunker()\
    .setInputCols(["sentence","ner"])\
    .setOutputCol("ner_chunk")\
    .setRegexParsers(["<DRUG>.*<FREQUENCY>"])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_chunker])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_chunker_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]

In [0]:
posology_ner.getClasses()

Out[52]: ['O',
 'B-DOSAGE',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-ROUTE',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'B-DRUG',
 'I-DRUG',
 'B-FORM',
 'I-DOSAGE',
 'B-DURATION',
 'I-DURATION',
 'I-FORM',
 'I-ROUTE']

In [0]:
light_model = LightPipeline(ner_chunker_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

list(zip(light_result['token'], light_result['ner']))

Out[53]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('fro', 'O'),
 ('3', 'O'),
 ('months', 'O'),
 ('.', 'O')]

In [0]:
light_result["ner_chunk"]

Out[54]: ['insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day']

#Chunk Filterer
ChunkFilterer will allow you to filter out named entities by some conditions or predefined look-up lists, so that you can feed these entities to other annotators like Assertion Status or Entity Resolvers. It can be used with two criteria: isin and regex.

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter()\
      .setInputCols(["sentence","token","ner"])\
      .setOutputCol("ner_chunk")
      
chunk_filterer = ChunkFilterer()\
      .setInputCols("sentence","ner_chunk")\
      .setOutputCol("chunk_filtered")\
      .setCriteria("isin")\
      .setWhiteList(['Advil','metformin', 'insulin lispro'])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter,
    chunk_filterer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chunk_filter_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]

In [0]:
light_model = LightPipeline(chunk_filter_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

light_result.keys()

Out[56]: dict_keys(['document', 'ner_chunk', 'chunk_filtered', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
light_result['ner_chunk'] 

Out[57]: ['1',
 'capsule',
 'Advil',
 'for 5 days',
 '40 units',
 'insulin glargine',
 'at night',
 '12 units',
 'insulin lispro',
 'with meals',
 'metformin',
 '1000 mg',
 'two times a day',
 'SGLT2 inhibitors']

In [0]:
light_result["chunk_filtered"]

Out[58]: ['Advil', 'insulin lispro', 'metformin']

In [0]:
ner_model = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
      .setInputCols("sentence","token","embeddings")\
      .setOutputCol("ner")
      
chunk_filterer = ChunkFilterer()\
      .setInputCols("sentence","ner_chunk")\
      .setOutputCol("chunk_filtered")\
      .setCriteria("isin")\
      .setWhiteList(['severe fever','sore throat'])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ner_model,
    ner_converter,
    chunk_filterer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chunk_filter_model = nlpPipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]

In [0]:
text = 'Patient with severe fever, severe cough, sore throat, stomach pain, and a headache.'

filter_df = spark.createDataFrame([[text]]).toDF('text')

chunk_filter_result = chunk_filter_model.transform(filter_df)

In [0]:
chunk_filter_result.select('ner_chunk.result', 'chunk_filtered.result').show(truncate=False)

+-------------------------------------------------------------------+---------------------------+
result |result |
+-------------------------------------------------------------------+---------------------------+
[severe fever, severe cough, sore throat, stomach pain, a headache]|[severe fever, sore throat]|
+-------------------------------------------------------------------+---------------------------+

## Changing entity labels with `NerConverterInternal()`

In [0]:
replace_dict = """Drug_BrandName, Drug
Frequency, Drug_Frequency
Dosage, Drug_Dosage
Strength, Drug_Strength
"""
with open('replace_dict.csv', 'w') as f:
    f.write(replace_dict)

In [0]:
%fs ls file:/databricks/driver

path,name,size
file:/databricks/driver/preload_class.lst,preload_class.lst,912170
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/replace_dict.csv,replace_dict.csv,91
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/logs/,logs/,4096


In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
  .setInputCols(["document"]) \
  .setOutputCol("sentence") 

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

jsl_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("jsl_ner")

jsl_ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "jsl_ner"]) \
  .setOutputCol("jsl_ner_chunk")

jsl_ner_converter_internal = NerConverterInternal()\
  .setInputCols(["sentence","token","jsl_ner"])\
  .setOutputCol("replaced_ner_chunk")\
  .setReplaceDictResource("file:/databricks/driver/replace_dict.csv","text", {"delimiter":","})
      
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter,
    jsl_ner_converter_internal
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_converter_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
import pandas as pd

def get_clinical_entities_light (light_model, text, chunk_name="ner_chunk"):

  light_result = light_model.fullAnnotate(text)

  chunks = []
  entities = []

  for n in light_result[0][chunk_name]:
          
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      
  df = pd.DataFrame({'chunks':chunks, 'entities':entities})

  return df

In [0]:
text ='The patient was prescribed 1 capsule of Parol with meals . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_model = LightPipeline(ner_converter_model)

jsl_ner_chunk_df = get_clinical_entities_light (light_model, text, chunk_name='jsl_ner_chunk')
replaced_ner_chunk_df = get_clinical_entities_light (light_model, text, chunk_name='replaced_ner_chunk')
pd.concat([jsl_ner_chunk_df, replaced_ner_chunk_df.iloc[:,1:].rename(columns= {'entities':'replaced'})], axis=1)


Out[5]:

,chunks,entities,replaced
0,1 capsule,Dosage,Drug_Dosage
1,Parol,Drug_BrandName,Drug
2,He,Gender,Gender
3,endocrinology service,Clinical_Dept,Clinical_Dept
4,she,Gender,Gender
5,discharged,Admission_Discharge,Admission_Discharge
6,40 units,Dosage,Drug_Dosage
7,insulin glargine,Drug_Ingredient,Drug_Ingredient
8,at night,Frequency,Drug_Frequency
9,12 units,Dosage,Drug_Dosage


## Training a Clinical NER (NCBI Disease Dataset)

**WARNING:** You should use TensorFlow version 2.3 for training a Clinical Ner.

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltrain.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltest.txt
  
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltest.txt", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltrain.txt", "dbfs:/")

Out[4]: True

In [0]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, 'file:/databricks/driver/NER_NCBIconlltrain.txt')

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
conll_data.count()

Out[6]: 3266

In [0]:
from pyspark.sql import functions as F

conll_data.select(F.explode(F.arrays_zip(conll_data.token.result, conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)

+------------+-----+
ground_truth|count|
+------------+-----+
O |75093|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
conll_data.select("label.result").distinct().count()

Out[8]: 1537

In [0]:
import pyspark.sql.functions as F

#conll_data.select(F.countDistinct("label.result")).show()
#conll_data.groupBy("label.result").count().show(truncate=False)

conll_data = conll_data.withColumn('unique', F.array_distinct(conll_data.label.result))\
                       .withColumn('c', F.size('unique'))\
                       .filter(F.col('c')>1)

conll_data.select(F.explode(F.arrays_zip(conll_data.token.result, conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)

+------------+-----+
ground_truth|count|
+------------+-----+
O |39427|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
# Clinical word embeddings trained on PubMED dataset
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
test_data = CoNLL().readDataset(spark, "file:/databricks/driver/NER_NCBIconlltest.txt")

test_data = clinical_embeddings.transform(test_data)

test_data.write.parquet('dbfs/NER_NCBIconlltest.parquet')

In [0]:
%fs ls file:/databricks/driver

path,name,size
file:/databricks/driver/preload_class.lst,preload_class.lst,912170
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/NER_NCBIconlltrain.txt,NER_NCBIconlltrain.txt,1091488
file:/databricks/driver/logs/,logs/,4096
file:/databricks/driver/NER_NCBIconlltest.txt,NER_NCBIconlltest.txt,244358
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/eventlogs/,eventlogs/,4096


## MedicalNER Graph

**WARNING:** For training an NER model with custom graph, please use TensorFlow version 2.3

In [0]:
import tensorflow
from sparknlp_jsl.training import tf_graph

In [0]:
tensorflow.__version__

Out[13]: '2.3.0'

Firstly, we will create graph and log folder.

In [0]:
%fs mkdirs file:/dbfs/ner/ner_logs

res5: Boolean = true

In [0]:
%fs mkdirs file:/dbfs/ner/medical_ner_graphs

res6: Boolean = true

In [0]:
%fs ls file:/dbfs/ner

path,name,size
file:/dbfs/ner/medical_ner_graphs/,medical_ner_graphs/,4096
file:/dbfs/ner/ner_logs/,ner_logs/,4096


We created ner log and graph files. 
Now, we will create graph and fit the model.

In [0]:
tf_graph.print_model_params("ner_dl")
tf_graph.build("ner_dl", build_params={"embeddings_dim": 200, "nchars": 85, "ntags": 12, "is_medical": 1}, model_location="/dbfs/ner/medical_ner_graphs", model_filename="auto")

ner_dl parameters.
Parameter Required Default value Description
ntags yes - Number of tags.
embeddings_dim no 200 Embeddings dimension.
nchars no 100 Number of chars.
lstm_size no 128 Number of LSTM units.
gpu_device no 0 Device for training.
is_medical no 0 Build a Medical Ner graph.
/databricks/python/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
 warnings.warn(
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/sparknlp_jsl/_tf_graph_builders/tf2contrib/lstm_ops.py:596: Layer.add_variable (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
ner_dl graph exported to /dbfs/ner/medical_ner_graphs/blstm_12_200_128_85.pb

In [0]:
# for open source users
'''
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py

!pip -q install tensorflow==1.15.0

import create_graph

ntags = 3 # number of labels
embeddings_dim = 200
nchars =83

create_graph.create_graph(ntags, embeddings_dim, nchars)
'''

Out[19]: '\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py\n\n!pip -q install tensorflow==1.15.0\n\nimport create_graph\n\nntags = 3 # number of labels\nembeddings_dim = 200\nnchars =83\n\ncreate_graph.create_graph(ntags, embeddings_dim, nchars)\n'

In [0]:
%fs ls file:/dbfs/ner/medical_ner_graphs/

path,name,size
file:/dbfs/ner/medical_ner_graphs/blstm_12_200_128_85.pb,blstm_12_200_128_85.pb,1767991


In [0]:
nerTagger = MedicalNerApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(2)\
  .setBatchSize(64)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setValidationSplit(0.2)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)\
  .setTestDataset("/NER_NCBIconlltest.parquet")\
  .setOutputLogsPath('dbfs:/ner/ner_logs')\
  .setGraphFolder('dbfs:/ner/medical_ner_graphs') # control your graph folder

ner_pipeline = Pipeline(stages=[
          clinical_embeddings,
          nerTagger
 ])

In [0]:
# 2 epochs 39 sec
ner_model = ner_pipeline.fit(conll_data)

In [0]:
%sh cd /dbfs/ner/ && ls -la

total 16
drwxrwxrwx 2 root root 4096 Feb 24 15:09 .
drwxrwxrwx 2 root root 4096 Feb 24 14:43 ..
drwxrwxrwx 2 root root 4096 Feb 24 15:09 medical_ner_graphs
drwxrwxrwx 2 root root 4096 Feb 24 15:06 ner_logs

In [0]:
%sh cd /dbfs/ner/ner_logs && ls -lt

total 3
-rwxrwxrwx 1 root root 2283 Feb 24 15:12 MedicalNerApproach_73f4f0678ff4.log

In [0]:
%sh cat /dbfs/ner/ner_logs/MedicalNerApproach_73f4f0678ff4.log

Name of the selected graph: /databricks/driver/file:/local_disk0/tmp/sparknlp_tmp_5114306091936852699/blstm_12_200_128_85.pb
Training started - total epochs: 2 - lr: 0.001 - batch size: 64 - labels: 3 - chars: 82 - training examples: 1367


Epoch 1/2 started, lr: 0.001, dataset size: 1367


Epoch 1/2 - 11.71s - loss: 487.52103 - avg training loss: 20.313375 - batches: 24
Quality on validation dataset (20.0%), validation examples = 273
time to finish evaluation: 1.58s
Total validation loss: 72.9965	Avg validation loss: 9.1246
label tp fp fn prec rec f1
I-Disease 114 14 646 0.890625 0.15 0.25675678
B-Disease 176 63 432 0.7364017 0.28947368 0.41558442
tp: 290 fp: 77 fn: 1078 labels: 2
Macro-average prec: 0.81351334, rec: 0.21973684, f1: 0.3460127
Micro-average prec: 0.79019076, rec: 0.2119883, f1: 0.33429396
Quality on test dataset: 
time to finish evaluation: 2.00s
Total test loss: 87.4943	Avg test loss: 6.2496
label tp fp fn prec rec f1
I-Disease 116 24 673 0.82857144 0.14702155 0.2497309
B-Disease 171 93 537 0.64772725 0.24152543 0.35185185
tp: 287 fp: 117 fn: 1210 labels: 2
Macro-average prec: 0.73814934, rec: 0.19427349, f1: 0.3075919
Micro-average prec: 0.71039605, rec: 0.19171676, f1: 0.3019463


Epoch 2/2 started, lr: 9.950249E-4, dataset size: 1367


Epoch 2/2 - 9.70s - loss: 223.80539 - avg training loss: 9.325225 - batches: 24
Quality on validation dataset (20.0%), validation examples = 273
time to finish evaluation: 1.07s
Total validation loss: 63.9876	Avg validation loss: 7.9984
label tp fp fn prec rec f1
I-Disease 282 53 478 0.84179103 0.37105262 0.51506853
B-Disease 282 75 326 0.789916 0.46381578 0.58445597
tp: 564 fp: 128 fn: 804 labels: 2
Macro-average prec: 0.8158535, rec: 0.41743422, f1: 0.55228823
Micro-average prec: 0.8150289, rec: 0.4122807, f1: 0.54757285
Quality on test dataset: 
time to finish evaluation: 1.96s
Total test loss: 62.5071	Avg test loss: 4.4648
label tp fp fn prec rec f1
I-Disease 273 84 516 0.7647059 0.34600762 0.47643983
B-Disease 296 123 412 0.7064439 0.4180791 0.5252884
tp: 569 fp: 207 fn: 928 labels: 2
Macro-average prec: 0.7355749, rec: 0.38204336, f1: 0.5028935
Micro-average prec: 0.7332474, rec: 0.38009351, f1: 0.50065994

### Evaluate your model

In [0]:
pred_df = ner_model.stages[-1].transform(test_data)

In [0]:
pred_df.columns

Out[26]: ['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

In [0]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

evaler = NerDLMetrics(mode="full_chunk", dropO=True)

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label").cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+------+
 entity| tp| fp| fn|total|precision|recall| f1|
+-------+-----+-----+-----+-----+---------+------+------+
Disease|247.0|340.0|457.0|704.0| 0.4208|0.3509|0.3826|
+-------+-----+-----+-----+-----+---------+------+------+

+------------------+
 macro|
+------------------+
0.3826491092176607|
+------------------+

None
+------------------+
 micro|
+------------------+
0.3826491092176607|
+------------------+

None

In [0]:
evaler = NerDLMetrics(mode="partial_chunk_per_token", dropO=True)

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label").cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+------+---------+------+------+
 entity| tp| fp| fn| total|precision|recall| f1|
+-------+-----+----+-----+------+---------+------+------+
Disease|702.0|74.0|795.0|1497.0| 0.9046|0.4689|0.6177|
+-------+-----+----+-----+------+---------+------+------+

+------------------+
 macro|
+------------------+
0.6176858776946766|
+------------------+

None
+------------------+
 micro|
+------------------+
0.6176858776946766|
+------------------+

None

In [0]:
ner_model.stages[1].write().overwrite().save('dbfs:/databricks/driver/models/custom_Med_NER_2e')

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

loaded_ner_model = MedicalNerModel.load("dbfs:/databricks/driver/models/custom_Med_NER_2e")\
 .setInputCols(["sentence", "token", "embeddings"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        clinical_embeddings,
        loaded_ner_model,
        converter])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)

from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [0]:
text = "She has a metastatic breast cancer to lung"

def get_preds(text, light_model):

    result = light_model.fullAnnotate(text)[0]

    return [(i.result, i.metadata['entity']) for i in result['ner_span']]

get_preds(text, light_model)

Out[31]: [('metastatic', 'Disease'), ('cancer', 'Disease')]

End of Notebook #